In [1]:
import pandas as pd
from konlpy.tag import Mecab
from functools import *
from collections import Counter
import numpy as np
import re
from tqdm.auto import tqdm

In [2]:
def cleaning_korean(content):
    content = content.replace('\n\n','')
    content = content.replace('”','')
    content = ' '.join(content.split(' '))
    #클리닝1
    content = re.sub(r'... 기자', '', content)
    content = re.sub(r'...기자', '', content)

    #괄호안 제거 
    ''' 괄호안 내용 제거하기 '''
    pattern_bracket1 = r'\([^)]*\)'
    pattern_bracket2 = r'\{[^)]*\}'
    pattern_bracket3 = r'\<[^)]*\>'
    pattern_bracket4 = r'\[[^)]*\]'
    x = '이건 {괄호 안의 불필요한 정보를} 삭제하는 코드다.'
    content = re.sub(pattern=pattern_bracket1, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket2, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket3, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket4, repl='', string= content)
    content = ' '.join(content.split())

    #클리닝
    content = content.lower() #lower case
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    content = re.sub(pattern=pattern, repl='', string=content)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    content = re.sub(pattern=pattern, repl='', string=content)

    # 특수기호 특수
    content = re.sub('[-=+#/\?:^$@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》Δ△·“=◎<>▷]', '', content)
    content = re.sub('[一-龥]', '',content)

    #pattern = '[^\w\s]'         # 특수기호제거
    #content = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"]', '',str(content)) #removepunctuation
    content = re.sub(r"[^a-zA-Z0-9가-힣\.\s]","",content)
    content = re.sub(pattern=pattern, repl='', string=content)
    content = re.sub(r'\s+', ' ', content) #remove extra space
    content = re.sub(r'<[^>]+>','',content) #remove Html tags
    content = re.sub(r'\s+', ' ', content) #remove spaces
    content = re.sub(r"^\s+", '', content) #remove space from start
    content = re.sub(r'\s+$', '', content) #remove space from the end
    return content

In [3]:
# 뉴스 로드 
df_temp = pd.read_parquet('df_temp.pq')

# 기간 지정 및 제외 기타 제거 
df = df_temp
df_sm = df.loc[df.contents_clean != '']\
    .loc[df.y != '제외']\
    .loc[df.y != '기타']\
    .loc[df.time < '2021-04-28']\
    .loc[df.time > '2020-01-01']

df_sm.title = df_sm.title.apply(lambda x : cleaning_korean(x))

In [5]:
df_sm.y.value_counts()

에너지및온실가스       7708
사회공헌           6326
인적자원관리         5741
소유구조           4898
ESG거버넌스        2867
제품/서비스         2208
공급망관리          2044
사업장안전및보건       1704
내부통제/투명성       1492
소비자보호          1449
이사회구성과운영       1377
지역사회영향         1373
자원순환           1232
공정거래           1173
계열회사와의거래        519
주주가치보호          463
이익배분            416
감사기구            375
인권              263
환경관리체계          202
대기오염            167
공시              118
친환경제품/서비스개발      79
수질오염             79
생물다양성            49
화학물질배출           34
기타오염             22
토양오염              6
Name: y, dtype: int64

In [31]:
def filter_list_in_title(df_trg, cate , keywords):
    trg = cate
    df_trg = df_trg.loc[df_trg.y == trg]
    
    rows = []
    for k,item in df_trg.iterrows():
        row = item.title
        
        FLAG_OUT= False
        for keyword in keywords:
            if keyword not in row:
                FLAG_OUT = True
                break
                
        if FLAG_OUT:
            continue
        else:
            rows.append(item)
    df_in = pd.DataFrame(rows)
    return df_in

def filter_list_out_title(df_trg, trg , keyword_list ):
    print('find words is not in title')

    df_trg = df_trg.loc[df_trg.y == trg]
    
    rows = []
    for k,item in df_trg.iterrows():
        row = item.title
        FLAG_IN= False
        for keys in keyword_list:
            for key in keys:
                if key in row:
                    FLAG_IN = True
                    break
            if FLAG_IN:
                continue
            else:
                rows.append(item)
    df_out = pd.DataFrame(rows)
    df_out['from'] = 'x'
    return df_out

def get_in_df( df , trg , keyword_list):
    print('find words is in title')
    # 키워드는 list of list 이다. 
    df_trg = df.loc[df.y == trg]
    df_inlist = []
    for keys in keyword_list:
        dfres = filter_list_in_title(df_trg , trg , keys)
        if len(dfres) != 0:
            dfres['from'] = ','.join(keys)
            df_inlist.append(dfres)
            print( trg,keys, dfres.shape)
    return df_inlist

In [32]:
# 키워드 넣으면 제목에 그 키워드가 들어가 있는 뉴스를 선별해준다. 
trg = '인적자원관리'
keyword_list = [['사망'] , ['임금' , '협상'] , ['파업'] , ['채용', '근로기준']]

df_inlist = get_in_df( df_sm , trg , keyword_list)


find words is in title
인적자원관리 ['사망'] (29, 11)
인적자원관리 ['임금', '협상'] (47, 11)
인적자원관리 ['파업'] (244, 11)


In [33]:
df_out  = filter_list_out_title( df_sm , trg , keyword_list)

find words is not in title


In [34]:
df_out

,index,id,keyword,time,title,link,contents,y,contents_clean,words,from
10,10,6010,SK이노베이션,2020-05-14 20:01:59,코로나19 구직난에 비대면 채용 늘려달라,http://www.ggilbo.com/news/articleView.html?id...,IT·대기업 중심으로 비대면 채용 활발\n\n지역 中企 자금난에 화상면접 뒤처져\n...,인적자원관리,it대기업 중심으로 비대면 채용 활발지역 자금난에 화상면접 뒤처져지역인재 수도권 유...,"[기업, 중심, 채용, 지역, 자금난, 화상, 면접, 지역, 수도, 유출, 심화, ...",x
10,10,6010,SK이노베이션,2020-05-14 20:01:59,코로나19 구직난에 비대면 채용 늘려달라,http://www.ggilbo.com/news/articleView.html?id...,IT·대기업 중심으로 비대면 채용 활발\n\n지역 中企 자금난에 화상면접 뒤처져\n...,인적자원관리,it대기업 중심으로 비대면 채용 활발지역 자금난에 화상면접 뒤처져지역인재 수도권 유...,"[기업, 중심, 채용, 지역, 자금난, 화상, 면접, 지역, 수도, 유출, 심화, ...",x
10,10,6010,SK이노베이션,2020-05-14 20:01:59,코로나19 구직난에 비대면 채용 늘려달라,http://www.ggilbo.com/news/articleView.html?id...,IT·대기업 중심으로 비대면 채용 활발\n\n지역 中企 자금난에 화상면접 뒤처져\n...,인적자원관리,it대기업 중심으로 비대면 채용 활발지역 자금난에 화상면접 뒤처져지역인재 수도권 유...,"[기업, 중심, 채용, 지역, 자금난, 화상, 면접, 지역, 수도, 유출, 심화, ...",x
26,26,6403,대상,2020-05-14 18:39:55,기존노조약화 컨설팅 비용 13억 지급 유성기업 대표이사 ...,https://www.lawissue.co.kr/view.php?ud=2020051...,피고인들의 부당노동행위가 이른바 '유성기업 노조파괴 사건'이라는 명칭으로 ... 주...,인적자원관리,피고인들의 부당노동행위가 이른바 유성기업 노조파괴 사건이라는 명칭으로 ... 주장은...,"[피고인, 부당, 노동, 행위, 유성기업, 노조, 파괴, 사건, 명칭, 주장, 피고...",x
26,26,6403,대상,2020-05-14 18:39:55,기존노조약화 컨설팅 비용 13억 지급 유성기업 대표이사 ...,https://www.lawissue.co.kr/view.php?ud=2020051...,피고인들의 부당노동행위가 이른바 '유성기업 노조파괴 사건'이라는 명칭으로 ... 주...,인적자원관리,피고인들의 부당노동행위가 이른바 유성기업 노조파괴 사건이라는 명칭으로 ... 주장은...,"[피고인, 부당, 노동, 행위, 유성기업, 노조, 파괴, 사건, 명칭, 주장, 피고...",x
...,...,...,...,...,...,...,...,...,...,...,...
106104,106104,4593,카카오,2020-05-12 19:58:18,이 와중에 파업 재개하는 현대 노조가 임금인상 발목 내부 불만,https://www.fnnews.com/news/202005121759494184,"1년째 임단협 마무리 못해\n\n오늘부터 간부급 노조원 파업\n\n""코로나로 수주도...",인적자원관리,1년째 임단협 마무리 못해오늘부터 간부급 노조원 파업코로나로 수주도 힘든데 비판현대...,"[마무리, 오늘, 간부급, 노조원, 파업, 코로나, 수주, 비판, 현대, 중공업, ...",x
106108,106108,4650,한국금융지주,2020-05-12 19:13:24,ok금융그룹 5월 가정의 달 맞아 임직원 가족 사랑,https://www.tfmedia.co.kr/news/article.html?no...,"(조세금융신문=곽호성 기자) OK저축은행, OK캐피탈 등이 소속돼 있는 OK금융그룹...",인적자원관리,ok저축은행 ok캐피탈 등이 소속돼 있는 ok금융그룹은 코로나19 극복을 위한 기부...,"[저축, 은행, 캐피탈, 소속, 금융, 그룹, 코로나, 극복, 기부, 가정, 임직원...",x
106108,106108,4650,한국금융지주,2020-05-12 19:13:24,ok금융그룹 5월 가정의 달 맞아 임직원 가족 사랑,https://www.tfmedia.co.kr/news/article.html?no...,"(조세금융신문=곽호성 기자) OK저축은행, OK캐피탈 등이 소속돼 있는 OK금융그룹...",인적자원관리,ok저축은행 ok캐피탈 등이 소속돼 있는 ok금융그룹은 코로나19 극복을 위한 기부...,"[저축, 은행, 캐피탈, 소속, 금융, 그룹, 코로나, 극복, 기부, 가정, 임직원...",x
106108,106108,4650,한국금융지주,2020-05-12 19:13:24,ok금융그룹 5월 가정의 달 맞아 임직원 가족 사랑,https://www.tfmedia.co.kr/news/article.html?no...,"(조세금융신문=곽호성 기자) OK저축은행, OK캐피탈 등이 소속돼 있는 OK금융그룹...",인적자원관리,ok저축은행 ok캐피탈 등이 소속돼 있는 ok금융그룹은 코로나19 극복을 위한 기부...,"[저축, 은행, 캐피탈, 소속, 금융, 그룹, 코로나, 극복, 기부, 가정, 임직원...",x
